## Project 3: Linear Regression Modeling of California Home Prices
Last updated: Feb 4, 2018


## Instructions
In this project, we will work with the California Home Price dataset.  We will train a regression model to predict median home prices.  

First, we will go through this example.  Then you will make modifications and run the code, collecting results.

Learning Objectives
Students will gain additional expertise in the following:

RDDs, DataFrames, data preprocessing, feature engineering, model training, model evalulation

## Lab Exercises

1) Go through all code and fill in the missing cells. This will prep data, train a model, predict, and evaluate model fit.  Compute and report the Mean Squared Error (MSE) in a table at the very bottom, where all MSE values should be summarized.  
Show all work.

2) Repeat (1) with at least one additional feature from the original set.  
3) Repeat (1) with at least one engineered feature based on one or more variables from the original set.  
4) Repeat (1) but do Lasso Regression instead of Linear Regression. hint: change the parameter: `


### Data Source
StatLib---Datasets Archive  
http://lib.stat.cmu.edu/datasets/

houses.zip
These spatial data contain 20,640 observations on housing prices with 9 economic covariates. It appeared in Pace and Barry (1997), "Sparse Spatial Autoregressions", Statistics and Probability Letters. Submitted by Kelley Pace (kpace@unix1.sncc.lsu.edu). [9/Nov/99] (536 kbytes)



Data Description
This tutorial makes use of the California Housing data set. It appeared in a 1997 paper titled Sparse Spatial Autoregressions, written by Pace, R. Kelley and Ronald Barry and published in the Statistics and Probability Letters journal. The researchers built this data set by using the 1990 California census data.

The data contains one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

These spatial data contain 20,640 observations on housing prices with 9 economic variables

All the block groups with zero entries for the independent and dependent variables have been excluded from the data.

The Median house value is the dependent variable and will be assigned the role of the target variable in your ML model.



### Preprocessing (completed offline)

cadata_raw.txt contains a data description at the top, followed by data.

1. Separated data from header  
   cal_housing_data_raw.txt  contains only data  
   cal_housing_header.txt contains only text
2. Some values are in scientific notation.  
   Spacing is inconsistent (first 6 fields separated by 2 spaces. Lat/long separated by 1 space)  
   Ran the following in Python to format values

import pandas as pd

d = pd.read_csv('/home/ubuntu/projects/cal_housing_data_raw.txt', header=None, sep='  ')

d.columns=['v1','v2','v3','v4','v5','v6','v7','v8']  
d['latitude'] = d.v8.map(lambda l: l.split()[0])  
d['longitude'] = d.v8.map(lambda l: l.split()[1])  
d.latitude = d.latitude.map(lambda v: float(v))  
d.longitude = d.longitude.map(lambda v: float(v))  
d.to_csv('/home/ubuntu/projects/cal_housing_data_preproc.txt', index=False, header=False)


In [1]:
# import pyspark modules
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
from pyspark.mllib.linalg import DenseVector
from pyspark.ml.feature import StandardScaler
from pyspark.ml.regression import LinearRegression
import pandas as pd

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

In [2]:
# read text file, which is comma-separated
house = pd.read_csv('data/cal_housing_data_preproc_w_header.txt')

In [3]:
house.head()

,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
0,452600.0,8.3252,41.0,880.0,129.0,322.0,126.0,37.88,-122.23
1,358500.0,8.3014,21.0,7099.0,1106.0,2401.0,1138.0,37.86,-122.22
2,352100.0,7.2574,52.0,1467.0,190.0,496.0,177.0,37.85,-122.24
3,341300.0,5.6431,52.0,1274.0,235.0,558.0,219.0,37.85,-122.25
4,342200.0,3.8462,52.0,1627.0,280.0,565.0,259.0,37.85,-122.25


In [4]:
# cast to Spark dataframe
df = sqlCtx.createDataFrame(house)

In [5]:
df.count()

20640

In [6]:
df.show(5)

+------------------+-----------------+------------------+-----------+--------------+----------+----------+--------+---------+
|median_house_value|    median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|
+------------------+-----------------+------------------+-----------+--------------+----------+----------+--------+---------+
|          452600.0|           8.3252|              41.0|      880.0|         129.0|     322.0|     126.0|   37.88|  -122.23|
|          358500.0|           8.3014|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|   37.86|  -122.22|
|          352100.0|7.257399999999999|              52.0|     1467.0|         190.0|     496.0|     177.0|   37.85|  -122.24|
|          341300.0|           5.6431|              52.0|     1274.0|         235.0|     558.0|     219.0|   37.85|  -122.25|
|          342200.0|           3.8462|              52.0|     1627.0|         280.0|     565.0|     259.0|   37.85|  -

In [7]:
df.printSchema()

root
 |-- median_house_value: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [8]:
df.select(['median_house_value','median_income']).show(5)

+------------------+-----------------+
|median_house_value|    median_income|
+------------------+-----------------+
|          452600.0|           8.3252|
|          358500.0|           8.3014|
|          352100.0|7.257399999999999|
|          341300.0|           5.6431|
|          342200.0|           3.8462|
+------------------+-----------------+
only showing top 5 rows



In [9]:
# Descriptive statistics
df.select('households','median_house_value','median_income','total_bedrooms').describe().show()

+-------+------------------+------------------+-----------------+-----------------+
|summary|        households|median_house_value|    median_income|   total_bedrooms|
+-------+------------------+------------------+-----------------+-----------------+
|  count|             20640|             20640|            20640|            20640|
|   mean| 499.5396802325581|206855.81690891474|3.870671002906974|537.8980135658915|
| stddev|382.32975283161073|115395.61587441366|1.899821717945268|421.2479059431317|
|    min|               1.0|           14999.0|           0.4999|              1.0|
|    max|            6082.0|          500001.0|          15.0001|           6445.0|
+-------+------------------+------------------+-----------------+-----------------+



### Additional Preprocessing

We want to do three more things before training a model:  

1) Scale the response variable median_house_value, dividing by 100000 and saving into column median_house_value_final

In [10]:
df_2=df.withColumn('median_house_value_final',df.median_house_value/100000)

2) Feature Engineering

Add new feature:  rooms_per_household (hint: use *withColumn()* function)  

In [11]:
df_3=df_2.withColumn('rooms_per_household',df.total_rooms/df.households)
df_3.show(2)

+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+------------------------+-------------------+
|median_house_value|median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|median_house_value_final|rooms_per_household|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+------------------------+-------------------+
|          452600.0|       8.3252|              41.0|      880.0|         129.0|     322.0|     126.0|   37.88|  -122.23|                   4.526|  6.984126984126984|
|          358500.0|       8.3014|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|   37.86|  -122.22|                   3.585|  6.238137082601054|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+------------------------+-------------------

3) Select and standardize features

Re-order and select columns

In [12]:
dfs = df_3.select("median_house_value_final", 
              "total_bedrooms", 
              "population", 
              "households", 
              "median_income", 
              "rooms_per_household")

In [13]:
dfs.show(3)

+------------------------+--------------+----------+----------+-----------------+-------------------+
|median_house_value_final|total_bedrooms|population|households|    median_income|rooms_per_household|
+------------------------+--------------+----------+----------+-----------------+-------------------+
|                   4.526|         129.0|     322.0|     126.0|           8.3252|  6.984126984126984|
|                   3.585|        1106.0|    2401.0|    1138.0|           8.3014|  6.238137082601054|
|                   3.521|         190.0|     496.0|     177.0|7.257399999999999|  8.288135593220339|
+------------------------+--------------+----------+----------+-----------------+-------------------+
only showing top 3 rows



We want to standardize the features, but not the response variable. We can do this by converting the DF to an RDD and
applying map().

In [14]:
input_data = dfs.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

In [15]:
input_data.take(3)

[(4.526, DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841])),
 (3.585, DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381])),
 (3.521, DenseVector([190.0, 496.0, 177.0, 7.2574, 8.2881]))]

In [16]:
# create dataframe for modeling
df2 = sqlCtx.createDataFrame(input_data, ["label", "features"])

In [17]:
df2.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|4.526|[129.0,322.0,126....|
|3.585|[1106.0,2401.0,11...|
|3.521|[190.0,496.0,177....|
+-----+--------------------+
only showing top 3 rows



In [18]:
# Feature scaling

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler; this computes the mean, standard deviation of each feature
scaler = standardScaler.fit(df2)

# Transform the data in `df2` with the scaler
scaled_df = scaler.transform(df2)

# Inspect the result
scaled_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213]))]

In [19]:
type(scaled_df)

pyspark.sql.dataframe.DataFrame

Split data into train 80%, test 20% sets, using `seed=314`

In [20]:
training,test=scaled_df.randomSplit([0.8,0.2],seed=314)

Initialize the logistic regression object

In [21]:
lr = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)

Fit the model using the training data

In [22]:
lrModel = lr.fit(training)

For each datapoint in the test set, make a prediction (hint: apply `transform()` to the model).
You will want the returned object to be a dataframe

In [23]:
predicted=lrModel.transform(test)

/opt/spark-1.6.1-bin-hadoop2.6/python/pyspark/ml/regression.py:123: UserWarning: weights is deprecated. Use coefficients instead.
  warnings.warn("weights is deprecated. Use coefficients instead.")


In [24]:
predicted.show(3)

+-----+--------------------+--------------------+------------------+
|label|            features|     features_scaled|        prediction|
+-----+--------------------+--------------------+------------------+
|0.269|[543.0,1423.0,482...|[1.28902717933820...|1.6043857976084994|
|0.329|[386.0,436.0,213....|[0.91632502987945...|1.3133532831902404|
|0.396|[296.0,1228.0,289...|[0.70267411617699...|1.2867595070730435|
+-----+--------------------+--------------------+------------------+
only showing top 3 rows



Convert the dataframe to an rdd. Then select only the `prediction` and `label` fields (hint: use map())

In [25]:
predAndLab = predicted.rdd.map(lambda x: (x.prediction, x.label))

In [26]:
predAndLab.take(3)

[(1.6043857976084994, 0.269),
 (1.3133532831902404, 0.329),
 (1.2867595070730435, 0.396)]

Evaluate the model by computing Mean Squared Error (MSE), which is the average sum of squared differences between predicted and label. 

This can be computed in a single line using `reduce()`

In [27]:
MSE = predAndLab.map(lambda (pred, lab): (pred - lab)**2).reduce(lambda x, y: x + y) / predAndLab.count()

In [28]:
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 0.762778654801


# Part2. Repeat (1) with housing_median_age from the original set.

In [29]:
dfs2 = df_3.select("median_house_value_final", 
              "total_bedrooms", 
              "population", 
              "households", 
              "median_income", 
              "rooms_per_household",
                 "housing_median_age")
dfs2.show(3)

+------------------------+--------------+----------+----------+-----------------+-------------------+------------------+
|median_house_value_final|total_bedrooms|population|households|    median_income|rooms_per_household|housing_median_age|
+------------------------+--------------+----------+----------+-----------------+-------------------+------------------+
|                   4.526|         129.0|     322.0|     126.0|           8.3252|  6.984126984126984|              41.0|
|                   3.585|        1106.0|    2401.0|    1138.0|           8.3014|  6.238137082601054|              21.0|
|                   3.521|         190.0|     496.0|     177.0|7.257399999999999|  8.288135593220339|              52.0|
+------------------------+--------------+----------+----------+-----------------+-------------------+------------------+
only showing top 3 rows



In [30]:
input_data2 = dfs2.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
input_data2.take(3)

[(4.526, DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 41.0])),
 (3.585, DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 21.0])),
 (3.521, DenseVector([190.0, 496.0, 177.0, 7.2574, 8.2881, 52.0]))]

In [31]:
# create dataframe for modeling
df22 = sqlCtx.createDataFrame(input_data2, ["label", "features"])
df22.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|4.526|[129.0,322.0,126....|
|3.585|[1106.0,2401.0,11...|
|3.521|[190.0,496.0,177....|
+-----+--------------------+
only showing top 3 rows



In [32]:
# Feature scaling

# Initialize the `standardScaler`
standardScaler2 = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler; this computes the mean, standard deviation of each feature
scaler2 = standardScaler2.fit(df22)

# Transform the data in `df2` with the scaler
scaled_df2 = scaler2.transform(df22)

# Inspect the result
scaled_df2.take(2)



[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 41.0]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 3.2577])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 21.0]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 1.6686]))]

In [33]:
training2,test2=scaled_df2.randomSplit([0.8,0.2],seed=314)
lr2 = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel2 = lr2.fit(training2)
predicted2=lrModel2.transform(test2)
predicted2.show(3)
predAndLab2 = predicted2.rdd.map(lambda x: (x.prediction, x.label))
predAndLab2.take(3)
MSE2 = predAndLab2.map(lambda (pred, lab): (pred - lab)**2).reduce(lambda x, y: x + y) / predAndLab2.count()
print("Mean Squared Error = " + str(MSE2))


+-----+--------------------+--------------------+------------------+
|label|            features|     features_scaled|        prediction|
+-----+--------------------+--------------------+------------------+
|0.269|[543.0,1423.0,482...|[1.28902717933820...| 1.604385529626804|
|0.329|[386.0,436.0,213....|[0.91632502987945...|1.3133528477522807|
|0.396|[296.0,1228.0,289...|[0.70267411617699...|1.2867590563333762|
+-----+--------------------+--------------------+------------------+
only showing top 3 rows

Mean Squared Error = 0.76277849851


# Part3: Repeat (1) with population_per_household from the original set.

In [34]:
df_4=df_3.withColumn('population_per_household',df.population/df.households)
df_4.show(2)


+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+------------------------+-------------------+------------------------+
|median_house_value|median_income|housing_median_age|total_rooms|total_bedrooms|population|households|latitude|longitude|median_house_value_final|rooms_per_household|population_per_household|
+------------------+-------------+------------------+-----------+--------------+----------+----------+--------+---------+------------------------+-------------------+------------------------+
|          452600.0|       8.3252|              41.0|      880.0|         129.0|     322.0|     126.0|   37.88|  -122.23|                   4.526|  6.984126984126984|      2.5555555555555554|
|          358500.0|       8.3014|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|   37.86|  -122.22|                   3.585|  6.238137082601054|       2.109841827768014|
+------------------+-------------+------

In [35]:
dfs3 = df_4.select("median_house_value_final", 
              "total_bedrooms", 
              "population", 
              "households", 
              "median_income", 
              "rooms_per_household",
                 "population_per_household")
dfs3.show(3)

+------------------------+--------------+----------+----------+-----------------+-------------------+------------------------+
|median_house_value_final|total_bedrooms|population|households|    median_income|rooms_per_household|population_per_household|
+------------------------+--------------+----------+----------+-----------------+-------------------+------------------------+
|                   4.526|         129.0|     322.0|     126.0|           8.3252|  6.984126984126984|      2.5555555555555554|
|                   3.585|        1106.0|    2401.0|    1138.0|           8.3014|  6.238137082601054|       2.109841827768014|
|                   3.521|         190.0|     496.0|     177.0|7.257399999999999|  8.288135593220339|      2.8022598870056497|
+------------------------+--------------+----------+----------+-----------------+-------------------+------------------------+
only showing top 3 rows



In [36]:
input_data3 = dfs3.rdd.map(lambda x: (x[0], DenseVector(x[1:])))
input_data3.take(3)

[(4.526, DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556])),
 (3.585, DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098])),
 (3.521, DenseVector([190.0, 496.0, 177.0, 7.2574, 8.2881, 2.8023]))]

In [37]:
# create dataframe for modeling
df23 = sqlCtx.createDataFrame(input_data3, ["label", "features"])
df23.show(3)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|4.526|[129.0,322.0,126....|
|3.585|[1106.0,2401.0,11...|
|3.521|[190.0,496.0,177....|
+-----+--------------------+
only showing top 3 rows



In [38]:
# Feature scaling

# Initialize the `standardScaler`
standardScaler3 = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler; this computes the mean, standard deviation of each feature
scaler3 = standardScaler3.fit(df23)

# Transform the data in `df2` with the scaler
scaled_df3 = scaler3.transform(df23)

# Inspect the result
scaled_df3.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031]))]

In [39]:
training3,test3=scaled_df3.randomSplit([0.8,0.2],seed=314)
lr3 = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel3 = lr3.fit(training3)
predicted3=lrModel3.transform(test3)
predicted3.show(3)
predAndLab3 = predicted3.rdd.map(lambda x: (x.prediction, x.label))
predAndLab3.take(3)
MSE3 = predAndLab3.map(lambda (pred, lab): (pred - lab)**2).reduce(lambda x, y: x + y) / predAndLab3.count()
print("Mean Squared Error = " + str(MSE3))

+-----+--------------------+--------------------+------------------+
|label|            features|     features_scaled|        prediction|
+-----+--------------------+--------------------+------------------+
|0.269|[543.0,1423.0,482...|[1.28902717933820...|1.6043858032617577|
|0.329|[386.0,436.0,213....|[0.91632502987945...| 1.313353292376104|
|0.396|[296.0,1228.0,289...|[0.70267411617699...| 1.286759516581707|
+-----+--------------------+--------------------+------------------+
only showing top 3 rows

Mean Squared Error = 0.762778658098


# Part4:  Repeat (1) but do Lasso Regression instead of Linear Regression. hint: change the parameter: regType

In [44]:
training,test=scaled_df.randomSplit([0.8,0.2],seed=314)
lr4 = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=1.0)

In [45]:
lrModel4 = lr4.fit(training)
predicted4=lrModel4.transform(test)
predicted4.show(3)
predAndLab4 = predicted4.rdd.map(lambda x: (x.prediction, x.label))
predAndLab4.take(3)
MSE4 = predAndLab4.map(lambda (pred, lab): (pred - lab)**2).reduce(lambda x, y: x + y) / predAndLab4.count()
print("Mean Squared Error = " + str(MSE4))

+-----+--------------------+--------------------+------------------+
|label|            features|     features_scaled|        prediction|
+-----+--------------------+--------------------+------------------+
|0.269|[543.0,1423.0,482...|[1.28902717933820...|1.6329494455112195|
|0.329|[386.0,436.0,213....|[0.91632502987945...|1.3597657687806006|
|0.396|[296.0,1228.0,289...|[0.70267411617699...|1.3348029719179593|
+-----+--------------------+--------------------+------------------+
only showing top 3 rows

Mean Squared Error = 0.780439463658


Show all RMSE values in a table at bottom, indicating run1, run2, run3, run4

In [42]:
import pandas as pd
out = pd.DataFrame(columns=['MSE'],index=['run1','run2','run3','run4'])


In [ ]:
out.iloc[0]['MSE']=0.762778654801;
out.iloc[1]['MSE']=0.76277849851;
out.iloc[2]['MSE']=0.762778658098;
out.iloc[2]['MSE']=0.780439463658;
out